<center>
    <img src="https://gitlab.com/ibm/skills-network/courses/placeholder101/-/raw/master/labs/module%201/images/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Lab 2: Data wrangling


Estimated time needed: **60** minutes


In this lab, we will perform some Exploratory Data Analysis (EDA) to find some patterns in the data and determine what would be the label for training supervised models.

In the data set, there are several different cases where the booster did not land successfully. Sometimes a landing was attempted but failed due to an accident; for example, <code>True Ocean</code> means the mission outcome was successfully  landed to a specific region of the ocean while <code>False Ocean</code> means the mission outcome was unsuccessfully landed to a specific region of the ocean. <code>True RTLS</code> means the mission outcome was successfully  landed to a ground pad <code>False RTLS</code> means the mission outcome was unsuccessfully landed to a ground pad.<code>True ASDS</code> means the mission outcome was successfully landed on  a drone ship <code>False ASDS</code> means the mission outcome was unsuccessfully landed on a drone ship.

In this lab we will mainly convert those outcomes into Training Labels with `1` means the booster successfully landed `0` means it was unsuccessful.


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing\_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


## Objectives

Perform exploratory  Data Analysis and determine Training Labels

*   Exploratory Data Analysis
*   Determine Training Labels


***


## Import Libraries and Define Auxiliary Functions


We will import the following libraries.


In [139]:
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
#NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np

### Data Analysis


Load Space X dataset, from last section.

In [133]:
sh = pd.read_csv('SpaceMissions.csv')
sx=pd.read_csv("spacex_web_scraped.csv")
sites = pd.read_csv('launch_site.csv')

### It is appropriate I believe to create a DataFrame about Launch Site so I can included the Latitude and Longitude

In [163]:
columns = ['Launch Site', 'Lat', 'Long']
site = pd.DataFrame(columns = columns)

la = pd.Series(['Cape Canaveral','28.396837','-80.605659'], index = site.columns)
lb = pd.Series(['KSC LC-39A', '28.608954234362187','-80.60429756306883'], index = site.columns)
lc = pd.Series(['VSFB', '34.445388', '-120.31541'], index = site.columns)
ld = pd.Series(['CCAFS LC-40','28.562302','-80.577356'], index = site.columns)
le = pd.Series(['CCAFS SLC-40','28.563197','-80.576820'], index = site.columns)
lf = pd.Series(['VAFB SLC-4E','34.632834','-120.610746'], index = site.columns)
site =site.append(la, ignore_index=True).append(lb, ignore_index=True).append(lc, ignore_index=True).append(ld, ignore_index=True).append(le, ignore_index=True).append(lf, ignore_index=True).append(lm, ignore_index=True)

In [164]:
site.to_csv('launch_sites.csv')

Identify and calculate the percentage of the missing values in each attribute


In [83]:
sx.isnull().sum()/sx.count()*100

Flight No.          0.000000
Launch site         0.000000
Payload             2.290076
Payload Mass(kg)    0.000000
Orbit               0.751880
Customer            0.751880
Launch outcome      0.000000
Version Booster     0.000000
Booster landing     0.000000
Date                0.000000
Time                0.000000
dtype: float64

Identify which columns are numerical and categorical:


In [84]:
sx.infer_objects()
sx.dtypes

Flight No.            int64
Launch site          object
Payload              object
Payload Mass(kg)    float64
Orbit                object
Customer             object
Launch outcome       object
Version Booster      object
Booster landing      object
Date                 object
Time                 object
dtype: object

### TASK 1: Calculate the number of launches on each site

The data contains several Space X  launch facilities: <a href='https://en.wikipedia.org/wiki/List_of_Cape_Canaveral_and_Merritt_Island_launch_sites?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01'>Cape Canaveral Space</a> Launch Complex 40  <b>VAFB SLC 4E </b> , Vandenberg Air Force Base Space Launch Complex 4E <b>(SLC-4E)</b>, Kennedy Space Center Launch Complex 39A <b>KSC LC 39A </b>.The location of each Launch Is placed in the column <code>LaunchSite</code>


Next, let's see the number of launches for each site.

Use the method  <code>value_counts()</code> on the column <code>LaunchSite</code> to determine the number of launches  on each site:


In [85]:
# Apply value_counts() on column LaunchSite
sx['Launch site'].value_counts()

CCAFS             39
KSC               38
Cape Canaveral    20
CCSFS             18
VAFB              16
VSFB               3
Name: Launch site, dtype: int64

Each launch aims to an dedicated orbit, and here are some common orbit types:


*   <b>LEO</b>: Low Earth orbit (LEO)is an Earth-centred orbit with an altitude of 2,000 km (1,200 mi) or less (approximately one-third of the radius of Earth),\[1] or with at least 11.25 periods per day (an orbital period of 128 minutes or less) and an eccentricity less than 0.25.\[2] Most of the manmade objects in outer space are in LEO <a href='https://en.wikipedia.org/wiki/Low_Earth_orbit?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01'>\[1]</a>.

*   <b>VLEO</b>: Very Low Earth Orbits (VLEO) can be defined as the orbits with a mean altitude below 450 km. Operating in these orbits can provide a number of benefits to Earth observation spacecraft as the spacecraft operates closer to the observation<a href='https://www.researchgate.net/publication/271499606_Very_Low_Earth_Orbit_mission_concepts_for_Earth_Observation_Benefits_and_challenges?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01'>\[2]</a>.

*   <b>GTO</b> A geosynchronous orbit is a high Earth orbit that allows satellites to match Earth's rotation. Located at 22,236 miles (35,786 kilometers) above Earth's equator, this position is a valuable spot for monitoring weather, communications and surveillance. Because the satellite orbits at the same speed that the Earth is turning, the satellite seems to stay in place over a single longitude, though it may drift north to south,” NASA wrote on its Earth Observatory website <a  href="https://www.space.com/29222-geosynchronous-orbit.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01" >\[3] </a>.

*   <b>SSO (or SO)</b>: It is a Sun-synchronous orbit  also called a heliosynchronous orbit is a nearly polar orbit around a planet, in which the satellite passes over any given point of the planet's surface at the same local mean solar time <a href="https://en.wikipedia.org/wiki/Sun-synchronous_orbit?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">\[4] <a>.

*   <b>ES-L1 </b>:At the Lagrange points the gravitational forces of the two large bodies cancel out in such a way that a small object placed in orbit there is in equilibrium relative to the center of mass of the large bodies. L1 is one such point between the sun and the earth <a href="https://en.wikipedia.org/wiki/Lagrange_point?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01#L1_point">\[5]</a> .

*   <b>HEO</b> A highly elliptical orbit, is an elliptic orbit with high eccentricity, usually referring to one around Earth <a href="https://en.wikipedia.org/wiki/Highly_elliptical_orbit?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">\[6]</a>.

*   <b> ISS </b> A modular space station (habitable artificial satellite) in low Earth orbit. It is a multinational collaborative project between five participating space agencies: NASA (United States), Roscosmos (Russia), JAXA (Japan), ESA (Europe), and CSA (Canada)<a href="https://en.wikipedia.org/wiki/International_Space_Station?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01"> \[7] </a>

*   <b> MEO </b> Geocentric orbits ranging in altitude from 2,000 km (1,200 mi) to just below geosynchronous orbit at 35,786 kilometers (22,236 mi). Also known as an intermediate circular orbit. These are "most commonly at 20,200 kilometers (12,600 mi), or 20,650 kilometers (12,830 mi), with an orbital period of 12 hours <a href="https://en.wikipedia.org/wiki/List_of_orbits?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01"> \[8] </a>

*   <b> HEO </b> Geocentric orbits above the altitude of geosynchronous orbit (35,786 km or 22,236 mi) <a href="https://en.wikipedia.org/wiki/List_of_orbits?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01"> \[9] </a>

*   <b> GEO </b> It is a circular geosynchronous orbit 35,786 kilometres (22,236 miles) above Earth's equator and following the direction of Earth's rotation <a href="https://en.wikipedia.org/wiki/Geostationary_orbit?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01"> \[10] </a>

*   <b> PO </b> It is one type of satellites in which a satellite passes above or nearly above both poles of the body being orbited (usually a planet such as the Earth <a href="https://en.wikipedia.org/wiki/Polar_orbit?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01"> \[11] </a>

some are shown in the following plot:


<div>
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/Orbits.png" width="500"/>
</div> 


### TASK 2: Calculate the number and occurrence of each orbit


Use the method  <code>.value_counts()</code> to determine the number and occurrence of each orbit in the  column <code>Orbit</code>


In [86]:
sx['Orbit'].value_counts()

LEO             75
GTO             34
SSO              9
Polar            7
MEO              4
Polar orbit      1
HEO              1
Sub-orbital      1
Heliocentric     1
Name: Orbit, dtype: int64

### TASK 3: Calculate the number and occurence of mission outcome per orbit type


Use the method <code>.value_counts()</code> on the column <code>Outcome</code> to determine the number of <code>landing_outcomes</code>.Then assign it to a variable landing_outcomes.


In [99]:
sx.loc[sx['Booster landing']=='Failure ']

,Flight No.,Launch site,Payload,Payload Mass(kg),Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time


In [100]:
landing_outcomes = sx['Booster landing'].value_counts()
landing_outcomes

Success         93
No attempt      22
Failure         11
Controlled       5
Uncontrolled     2
Precluded        1
Name: Booster landing, dtype: int64

<code>True Ocean</code> means the mission outcome was successfully  landed to a specific region of the ocean while <code>False Ocean</code> means the mission outcome was unsuccessfully landed to a specific region of the ocean. <code>True RTLS</code> means the mission outcome was successfully  landed to a ground pad <code>False RTLS</code> means the mission outcome was unsuccessfully landed to a ground pad.<code>True ASDS</code> means the mission outcome was successfully  landed to a drone ship <code>False ASDS</code> means the mission outcome was unsuccessfully landed to a drone ship. <code>None ASDS</code> and <code>None None</code> these represent a failure to land.


In [101]:
for i,outcome in enumerate(landing_outcomes.keys()):
    print(i,outcome)

0 Success
1 No attempt
2 Failure
3 Controlled
4 Uncontrolled
5 Precluded


We create a set of outcomes where the second stage did not land successfully:


In [102]:
bad_outcomes=set(landing_outcomes.keys()[[1,2,4,5,]])
bad_outcomes

{'Failure', 'No attempt', 'Precluded', 'Uncontrolled'}

### TASK 4: Create a landing outcome label from Outcome column


Using the <code>Outcome</code>,  create a list where the element is zero if the corresponding  row  in  <code>Outcome</code> is in the set <code>bad_outcome</code>; otherwise, it's one. Then assign it to the variable <code>landing_class</code>:


In [103]:
# landing_class = 0 if bad_outcome
# landing_class = 1 otherwise
olist = sx['Booster landing'].to_list()
landing_class = []
for x, outcome in enumerate(olist):
    if outcome in bad_outcomes:
        landing_class.append(0)
    else:
        landing_class.append(1)

This variable will represent the classification variable that represents the outcome of each launch. If the value is zero, the  first stage did not land successfully; one means  the first stage landed Successfully


In [104]:
sx['Class']=landing_class
sx[['Class']].head(20)

,Class
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,1
9,1


In [105]:
sx.head(10)

,Flight No.,Launch site,Payload,Payload Mass(kg),Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time,Class
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Success,F9 v1.0B0003,Failure,2010-06-04,18:45,0
1,2,CCAFS,Dragon,0.0,LEO,NASA,Success,F9 v1.0B0004,Failure,2010-12-08,15:43,0
2,3,CCAFS,Dragon,525.0,LEO,NASA,Success,F9 v1.0B0005,No attempt,2012-05-22,07:44,0
3,4,CCAFS,SpaceX CRS-1,4700.0,LEO,NASA,Success,F9 v1.0B0006,No attempt,2012-10-08,00:35,0
4,5,CCAFS,SpaceX CRS-2,4877.0,LEO,NASA,Success,F9 v1.0B0007,No attempt,2013-03-01,15:10,0
5,6,VAFB,CASSIOPE,500.0,Polar orbit,MDA,Success,F9 v1.1B1003,Uncontrolled,2013-09-29,16:00,0
6,7,CCAFS,SES-8,3170.0,GTO,SES,Success,F9 v1.1,No attempt,2013-12-03,22:41,0
7,8,CCAFS,Thaicom 6,3325.0,GTO,Thaicom,Success,F9 v1.1,No attempt,2014-01-06,22:06,0
8,9,Cape Canaveral,SpaceX CRS-3,2296.0,LEO,NASA,Success,F9 v1.1,Controlled,2014-04-18,19:25,1
9,10,Cape Canaveral,Orbcomm-OG2,1316.0,LEO,Orbcomm,Success,F9 v1.1,Controlled,2014-07-14,15:15,1


We can use the following line of code to determine  the success rate:


In [106]:
sx["Class"].mean()

0.7313432835820896

We can now export it to a CSV for the next section,but to make the answers consistent, in the next lab we will provide data in a pre-selected date range.


In [107]:
sx.to_csv("booster_outcome.csv", index=False)

In [108]:
fundf = sx

In [109]:
fundf.head(20)

,Flight No.,Launch site,Payload,Payload Mass(kg),Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time,Class
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Success,F9 v1.0B0003,Failure,2010-06-04,18:45,0
1,2,CCAFS,Dragon,0.0,LEO,NASA,Success,F9 v1.0B0004,Failure,2010-12-08,15:43,0
2,3,CCAFS,Dragon,525.0,LEO,NASA,Success,F9 v1.0B0005,No attempt,2012-05-22,07:44,0
3,4,CCAFS,SpaceX CRS-1,4700.0,LEO,NASA,Success,F9 v1.0B0006,No attempt,2012-10-08,00:35,0
4,5,CCAFS,SpaceX CRS-2,4877.0,LEO,NASA,Success,F9 v1.0B0007,No attempt,2013-03-01,15:10,0
5,6,VAFB,CASSIOPE,500.0,Polar orbit,MDA,Success,F9 v1.1B1003,Uncontrolled,2013-09-29,16:00,0
6,7,CCAFS,SES-8,3170.0,GTO,SES,Success,F9 v1.1,No attempt,2013-12-03,22:41,0
7,8,CCAFS,Thaicom 6,3325.0,GTO,Thaicom,Success,F9 v1.1,No attempt,2014-01-06,22:06,0
8,9,Cape Canaveral,SpaceX CRS-3,2296.0,LEO,NASA,Success,F9 v1.1,Controlled,2014-04-18,19:25,1
9,10,Cape Canaveral,Orbcomm-OG2,1316.0,LEO,Orbcomm,Success,F9 v1.1,Controlled,2014-07-14,15:15,1


In [110]:
fundf['Payload Mass(kg)']= fundf['Payload Mass(kg)'].astype(float)
fundf.groupby('Class')['Payload Mass(kg)'].mean()

Class
0    4958.944444
1    8390.010204
Name: Payload Mass(kg), dtype: float64

In [18]:
fundf.groupby('LaunchSite')['Class'].value_counts(normalize = True).to_frame()

Class
LaunchSite   Class          
CCAFS SLC 40 1      0.600000
             0      0.400000
KSC LC 39A   1      0.772727
             0      0.227273
VAFB SLC 4E  1      0.769231
             0      0.230769

<code>df.to_csv("dataset_part\_2.csv", index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/joseph-s-50398b136/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD.


<a href="https://www.linkedin.com/in/nayefaboutayoun/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">Nayef Abou Tayoun</a> is a Data Scientist at IBM and pursuing a Master of Management in Artificial intelligence degree at Queen's University.


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By    | Change Description      |
| ----------------- | ------- | ------------- | ----------------------- |
| 2021-08-31        | 1.1     | Lakshmi Holla | Changed Markdown        |
| 2020-09-20        | 1.0     | Joseph        | Modified Multiple Areas |
| 2020-11-04        | 1.1.    | Nayef         | updating the input data |
| 2021-05-026       | 1.1.    | Joseph        | updating the input data |


Copyright © 2021 IBM Corporation. All rights reserved.
